#### 경도 위도 --> 주소 변환

In [2]:
import pandas as pd
import requests 
import sys
import json
import datetime 

In [3]:
def json_request(url='', encoding='utf-8', success=None, error=lambda e: print('%s : %s' % (e, datetime.now()), file=sys.stderr)):
    headers = {'Authorization': 'KakaoAK {}'.format(APP_KEY)}
    resp = requests.get(url, headers=headers)
    # print('%s : success for request [%s]' % (datetime.now(), url))
    return resp.text

In [4]:
def reverse_geocode(longitude, latitude):
    # 파라미터 최적화하여 url 생성
    url = '%s?x=%s&y=%s' %(URL, longitude, latitude)
    # json request
    try:
        # print('try')
        json_req = json_request(url=url)
        json_data = json.loads(json_req)
        json_doc = json_data.get('documents')[0]
        json_name = json_doc.get('address_name')
        json_code = json_doc.get('code')
    except:
        # print('nan')
        json_name = 'NaN'
        json_code = 'NaN'
    return json_name, json_code

In [5]:
def get_address(x,y):
    address = []        
    json_name, json_code = reverse_geocode(x, y) 
    address.append(json_name)
    return address

In [6]:
def get_code(x,y):
    code = []        
    json_name, json_code = reverse_geocode(x, y) 
    code.append(json_code)
    return code

In [7]:
APP_KEY = '4269e8dab7d1e5016e7d1e3eaa8aba03' 
URL = 'https://dapi.kakao.com/v2/local/geo/coord2regioncode.json'

df_all = pd.read_csv('역별위치_위도경도.csv') 
df = df_all[['line', 'name', 'lat', 'lng']] # 기관ID, 병원경도, 병원위도 열만 추출 (도로명주소는 일부러 날림)


for i in range(len(df)): 
    x_crd = float(df.loc[i, ['lng']])
    y_crd = float(df.loc[i, ['lat']])
    address = get_address(x_crd, y_crd)
    code = get_code(x_crd, y_crd)
    df.loc[i, ['ADDRESS']] = address
    df.loc[i, ['CODE']] = code
    
    if i%100 == 0:
        print('{} 번째 경위도 좌표계에 대한 정보 불러오는 중...'.format(i))

0 번째 경위도 좌표계에 대한 정보 불러오는 중...
100 번째 경위도 좌표계에 대한 정보 불러오는 중...
200 번째 경위도 좌표계에 대한 정보 불러오는 중...
300 번째 경위도 좌표계에 대한 정보 불러오는 중...
400 번째 경위도 좌표계에 대한 정보 불러오는 중...
500 번째 경위도 좌표계에 대한 정보 불러오는 중...
600 번째 경위도 좌표계에 대한 정보 불러오는 중...
700 번째 경위도 좌표계에 대한 정보 불러오는 중...


In [17]:
df.to_csv('지하철 역별위치_행정동 변환.csv', index=False)

#### 서울시 추출

In [196]:
data = pd.read_csv('지하철 역별위치_행정동 변환.csv')
data

,line,name,lat,lng,ADDRESS,CODE
0,01호선,녹양,37.759380,127.042292,경기도 의정부시 녹양동,4.115011e+09
1,01호선,남영,37.541021,126.971300,서울특별시 용산구 갈월동,1.117010e+09
2,01호선,용산,37.529849,126.964561,서울특별시 용산구 한강로3가,1.117013e+09
3,01호선,노량진,37.514219,126.942454,서울특별시 동작구 노량진동,1.159010e+09
4,01호선,대방,37.513342,126.926382,서울특별시 영등포구 신길동,1.156013e+09
...,...,...,...,...,...,...
725,인천선,지식정보단지,37.378384,126.645168,인천광역시 연수구 송도동,2.818511e+09
726,인천선,인천대입구,37.386007,126.639484,인천광역시 연수구 송도동,2.818511e+09
727,인천선,센트럴파크,37.393054,126.634729,인천광역시 연수구 송도동,2.818511e+09
728,인천선,국제업무지구,37.399907,126.630347,인천광역시 연수구 송도동,2.818511e+09


In [197]:
data['시'] = data.ADDRESS.str.split(' ').str[0]
data['구'] = data.ADDRESS.str.split(' ').str[1]
data['동'] = data.ADDRESS.str.split(' ').str[2]
data = data.drop(['lat', 'lng', 'ADDRESS', 'CODE'], axis=1)

In [199]:
data

,line,name,시,구,동
0,01호선,녹양,경기도,의정부시,녹양동
1,01호선,남영,서울특별시,용산구,갈월동
2,01호선,용산,서울특별시,용산구,한강로3가
3,01호선,노량진,서울특별시,동작구,노량진동
4,01호선,대방,서울특별시,영등포구,신길동
...,...,...,...,...,...
725,인천선,지식정보단지,인천광역시,연수구,송도동
726,인천선,인천대입구,인천광역시,연수구,송도동
727,인천선,센트럴파크,인천광역시,연수구,송도동
728,인천선,국제업무지구,인천광역시,연수구,송도동


In [214]:
seoul = data[data.시 == '서울특별시']
seoul.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 384 entries, 1 to 658
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   line    384 non-null    object
 1   name    384 non-null    object
 2   시       384 non-null    object
 3   구       384 non-null    object
 4   동       384 non-null    object
dtypes: object(5)
memory usage: 18.0+ KB


In [215]:
seoul = seoul.drop_duplicates(['name'])
seoul.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 292 entries, 1 to 658
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   line    292 non-null    object
 1   name    292 non-null    object
 2   시       292 non-null    object
 3   구       292 non-null    object
 4   동       292 non-null    object
dtypes: object(5)
memory usage: 13.7+ KB


In [216]:
seoul = seoul.drop('시', axis=1)

In [217]:
seoul.to_csv('지하철 역별위치_서울 행정동(중복 제거).csv', index=False)

In [192]:
dd = pd.read_csv('서울시 행정동.csv' ,encoding='cp949')
ddd = dd['행정동']
ddd

0       사직동
1       삼청동
2       부암동
3       평창동
4       무악동
       ... 
421    천호2동
422      길동
423     강일동
424    상일1동
425    상일2동
Name: 행정동, Length: 426, dtype: object

In [194]:
seoul[~seoul.동.isin(ddd)]

,line,name,구,동
2,03호선,신사,강남구,논현동
3,07호선,강남구청,강남구,논현동
4,07호선,학동,강남구,논현동
5,07호선,논현,강남구,논현동
6,분당선,강남구청,강남구,논현동
...,...,...,...,...
379,경춘선,망우,중랑구,상봉동
380,06호선,봉화산,중랑구,신내동
381,07호선,중화,중랑구,중화동
382,경의선,중랑,중랑구,중화동


In [190]:
seoul[seoul['동'].str.contains('동') == False]

,line,name,구,동


In [165]:
seoul.iloc[186:189, 3] = '충현동'
seoul.iloc[305:310, 3] = '한강로동'
seoul.iloc[333, 3] = '청운효자동'
seoul.iloc[337, 3] = '종로1.2.3.4가동'
seoul.iloc[338:341, 3] = '종로5.6가동'
seoul.iloc[352, 3] = '명동'
seoul.iloc[353:357, 3] = '을지로동'
seoul.iloc[357:360, 3] = '광희동'
seoul.iloc[361:364, 3] = '소공동'

seoul = seoul.sort_values(by=['구', '동'])
seoul = seoul.reset_index(drop=True)

#### 동별 정렬

In [107]:
sss = ss.groupby(by=['구', '동'], as_index=False).size()